# **Preprocessing**

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout,LeakyReLU
# Import pandas and read the charity_data.csv from the provided cloud URL.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN','NAME'])

In [3]:
# Determine the number of unique values in each column.
unique_count = application_df.nunique()
unique_count

,0
APPLICATION_TYPE,17
AFFILIATION,6
CLASSIFICATION,71
USE_CASE,5
ORGANIZATION,4
STATUS,2
INCOME_AMT,9
SPECIAL_CONSIDERATIONS,2
ASK_AMT,8747
IS_SUCCESSFUL,2


In [4]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_counts = application_df['APPLICATION_TYPE'].value_counts()
application_counts

,count
APPLICATION_TYPE,
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
T9,156


In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
type_threshold = 500
application_types_to_replace = list(application_counts[application_counts < type_threshold].index)

# Replace in dataframe
application_df['APPLICATION_TYPE'] =  application_df['APPLICATION_TYPE'].replace(application_types_to_replace, 'Other')

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()


,count
APPLICATION_TYPE,
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
Other,276


In [6]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_count = application_df['CLASSIFICATION'].value_counts()
classification_count

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
...,...
C1248,1
C6100,1
C1820,1


In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classificaton_count= classification_count.loc[classification_count > 1]

In [8]:
classification_threshold = 1000
classification_to_replace = list(classification_count[classification_count < classification_threshold].index)
application_df['CLASSIFICATION'] =  application_df['CLASSIFICATION'].replace(classification_to_replace, 'Other')
application_df['CLASSIFICATION'].value_counts()


,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
Other,2261
C3000,1918
C2100,1883


In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df, columns=['APPLICATION_TYPE', 'CLASSIFICATION','AFFILIATION','USE_CASE','ORGANIZATION','SPECIAL_CONSIDERATIONS', 'INCOME_AMT'], dtype=int)
application_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,SPECIAL_CONSIDERATIONS_Y,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,5000,1,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,1,108590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0


In [10]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL']
X = application_df.drop(columns='IS_SUCCESSFUL')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# **Compile, Train and Evaluate the Model**

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
def deep_neural_net(input_dim , first_layer_node_number, second_layer_node_number, first_activation, second_activation, third_activation):
    nn = tf.keras.models.Sequential()

# First hidden layer
    nn.add(tf.keras.layers.Dense(units= first_layer_node_number, activation= first_activation, input_dim=(input_dim)))
    nn.add(Dropout(0.2))

# Second hidden layer
    nn.add(tf.keras.layers.Dense(units= second_layer_node_number, activation= second_activation))

# Output layer
    nn.add(tf.keras.layers.Dense(units=1, activation= third_activation))
# Check the structure of the model
    nn.summary()
    return nn, first_layer_node_number, second_layer_node_number, first_activation, second_activation, third_activation



# **Deliverable Model 1**

In [13]:
optimization_1_model = deep_neural_net(len(X_train_scaled[0]),88, 20, 'relu', 'relu', 'sigmoid')
first_layer_nodes_number = optimization_1_model[1]
second_layer_node_number = optimization_1_model[2]
first_activation = optimization_1_model[3]
second_activation = optimization_1_model[4]
third_activation = optimization_1_model[5]
optimization_1_model[0].compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
fit_model = optimization_1_model[0].fit(X_train_scaled, y_train, epochs=100)
model_loss, model_accuracy = optimization_1_model[0].evaluate(X_test_scaled,y_test,verbose=2)
print("Delivery Model 1")
print("------------------------------------------------")
print(f"""Layer1:{first_layer_nodes_number} neurons, activation function:{first_activation},
Layer2:{second_layer_node_number} neurons, activation function:{second_activation} ,
Output Layer: 1 neuron, activation function:{third_activation}""")
print(f"""Loss: {model_loss},
Accuracy: {model_accuracy}""")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 88)                  │           3,872 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 88)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 20)                  │           1,780 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              21 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,673 (22.16 KB)

 Trainable params: 5,673 (22.16 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.6901 - loss: 0.6077
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7237 - loss: 0.5656
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7272 - loss: 0.5596
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7318 - loss: 0.5547
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7279 - loss: 0.5575
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7279 - loss: 0.5560
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7331 - loss: 0.5471
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7270 - loss: 0.5559
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7299 - loss: 0.5548
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7326 - loss: 0.5499
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7370 - loss: 0.5458
Epoch 12/100
804/804 ━━━━━━━━━━━━━━━━━━━━

In [14]:
# Export our model to HDF5 file
optimization_1_model[0].save("AlphabetSoupCharity_Optimization1.h5", save_format='h5')

# **Deliverable Model 2**

In [15]:
optimization_2_model = deep_neural_net(len(X_train_scaled[0]),90, 40, 'relu', LeakyReLU(alpha=0.1) ,'sigmoid')
optimization_2_model[0].compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
fit_model = optimization_2_model[0].fit(X_train_scaled, y_train, epochs=100)
model_loss, model_accuracy = optimization_2_model[0].evaluate(X_test_scaled,y_test,verbose=2)
print("Delivery Model 2")
print("------------------------------------------------")
print(f"""Layer1:{optimization_2_model[1]} neurons, activation function:{optimization_2_model[3]},
Layer2:{ optimization_2_model[2]} neurons, activation function:{optimization_2_model[4]} ,
Output Layer: 1 neuron, activation function:{optimization_2_model[5]}""")
print(f"""Loss: {model_loss},
Accuracy: {model_accuracy}""")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                      │ (None, 90)                  │           3,960 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 90)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 40)                  │           3,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │              41 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,641 (29.85 KB)

 Trainable params: 7,641 (29.85 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.6925 - loss: 0.6038
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7279 - loss: 0.5649
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7275 - loss: 0.5570
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7338 - loss: 0.5498
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7347 - loss: 0.5496
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7300 - loss: 0.5529
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7322 - loss: 0.5490
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7315 - loss: 0.5527
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7394 - loss: 0.5460
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7329 - loss: 0.5495
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7331 - loss: 0.5513
Epoch 12/100
804/804 ━━━━━━━━━━━━━━━━━━━━

In [16]:
# Export our model to HDF5 file
optimization_2_model[0].save("AlphabetSoupCharity_Optimization2.h5", save_format='h5')

# **Deliverable Model 3**

In [17]:
optimization_3_model = deep_neural_net(len(X_train_scaled[0]),80, 50,  LeakyReLU(alpha=0.1),  LeakyReLU(alpha=0.1), 'sigmoid')
optimization_3_model[0].compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
fit_model = optimization_3_model[0].fit(X_train_scaled, y_train, epochs=100)
model_loss, model_accuracy = optimization_3_model[0].evaluate(X_test_scaled,y_test,verbose=2)
print("Delivery Model 3")
print("------------------------------------------------")
print(f"""Layer1:{optimization_3_model[1]} neurons, activation function:{optimization_3_model[3]},
Layer2:{ optimization_3_model[2]} neurons, activation function:{optimization_3_model[4]} ,
Output Layer: 1 neuron, activation function:{optimization_3_model[5]}""")
print(f"""Loss: {model_loss},
Accuracy: {model_accuracy}""")

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                      │ (None, 80)                  │           3,520 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 80)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 50)                  │           4,050 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 1)                   │              51 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,621 (29.77 KB)

 Trainable params: 7,621 (29.77 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7062 - loss: 0.5974
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7210 - loss: 0.5728
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7288 - loss: 0.5563
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7301 - loss: 0.5553
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7311 - loss: 0.5522
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7304 - loss: 0.5539
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7326 - loss: 0.5522
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7298 - loss: 0.5524
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7358 - loss: 0.5471
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7364 - loss: 0.5476
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7315 - loss: 0.5506
Epoch 12/100
804/804 ━━━━━━━━━━━━━━━━━━━━

In [18]:
# Export our model to HDF5 file
optimization_3_model[0].save("AlphabetSoupCharity_Optimization3.h5", save_format='h5')